## 作業目標: 了解斷詞演算法的背後計算

### 根據課程講述的內容, 請計算出下列剩餘所有情況的
若有一個人連續觀察到三天水草都是乾燥的(Dry), 則這三天的天氣機率為何？(可參考講義第13頁)
(Hint: 共有8種可能機率)

```python
states = ('sunny', 'rainy')
observations = ('dry', 'damp', 'soggy')
start_probability = {'sunny': 0.4, 'rainy': 0.6}
transition_probability = {'sunny':{'sunny':0.6, 'rainy':0.4},
                          'rainy': {'sunny':0.3, 'rainy':0.7}}
emission_probatility = {'sunny': {'dry':0.6, 'damp':0.3, 'soggy':0.1},
                        'rainy': {'dry':0.1, 'damp':0.4, 'soggy':0.5}}
```

```
觀察狀態 = 'dry', 'dry', 'dry'
Sunny, Sunny, Sunny: 0.4*(0.6)*0.6*(0.6)*0.6*(0.6) = 0.031104
Rainy, Sunny, Sunny: 0.6*(0.1)*0.3*(0.6)*0.6*(0.6) = 0.003888
Sunny, Rainy, Sunny: (0.4*0.6)*(0.4*0.1)*(0.3*0.6) = 0.001728
Sunny, Sunny, Rainy: (0.4*0.6)*(0.6*0.6)*(0.4*0.1) = 0.003456
Rainy, Rainy, Sunny: (0.6*0.1)*(0.7*0.1)*(0.3*0.6) = 0.000756
Sunny, Rainy, Rainy: (0.4*0.6)*(0.4*0.1)*(0.7*0.1) = 0.000672
Rainy, Sunny, Rainy: (0.6*0.1)*(0.3*0.6)*(0.4*0.1) = 0.000432
Rainy, Rainy, Rainy: (0.6*0.1)*(0.7*0.1)*(0.7*0.1) = 0.000294


最大機率為: Sunny, Sunny, Sunny
```

### 根據上述條件, 寫出Viterbi應用程式

In [1]:
observations = ('dry', 'dry', 'dry') #實際上觀察到的狀態為dry, dry, dry
states = ('sunny', 'rainy')
start_probability = {'sunny': 0.4, 'rainy': 0.6}
transition_probability = {'sunny':{'sunny':0.6, 'rainy':0.4},
                          'rainy': {'sunny':0.3, 'rainy':0.7}}
emission_probatility = {'sunny': {'dry':0.6, 'damp':0.3, 'soggy':0.1},
                        'rainy': {'dry':0.1, 'damp':0.4, 'soggy':0.5}}

In [14]:
def viterbi(obs, states, start_p, trans_p, emit_p):
    V = [{}]
    path = {}

    # Initialize base cases (t == 0)
    for y in states:
        V[0][y] = start_p[y] * emit_p[y][obs[0]]
        path[y] = [y]
        
    # Run Viterbi for t > 0
    for t in range(1,len(obs)):
        V.append({})
        newpath = {}
        
        for cur_state in states:
            max_prob = -99999
            state = ''
            for pre_state in states:
                # 前一天狀態 * 轉移矩陣 * 發射矩陣 
                # 例如：dry的情況，將 sunny(前一天) * rainy(今天) 跟 sunny(前一天) * sunny(今天)，看哪個機率大 將狀態跟機率存起來          
                prob = V[t-1][pre_state] * trans_p[pre_state][cur_state] * emit_p[cur_state][obs[t]]
                if prob > max_prob:
                    max_prob = prob
                    state = pre_state
                    
            # 取得最大的            
            V[t][cur_state] = max_prob
            newpath[cur_state] = path[state] + [cur_state]

        # Don't need to remember the old paths
        path = newpath
        print(path)
        print(V)
    (prob, state) = max([(V[len(obs) - 1][final_state], final_state) for final_state in states])
    return (prob, path[state])

In [15]:
result = viterbi(observations,
                 states,
                 start_probability,
                 transition_probability,
                 emission_probatility)

{'sunny': ['sunny', 'sunny'], 'rainy': ['sunny', 'rainy']}
[{'sunny': 0.24, 'rainy': 0.06}, {'sunny': 0.08639999999999999, 'rainy': 0.009600000000000001}]
{'sunny': ['sunny', 'sunny', 'sunny'], 'rainy': ['sunny', 'sunny', 'rainy']}
[{'sunny': 0.24, 'rainy': 0.06}, {'sunny': 0.08639999999999999, 'rainy': 0.009600000000000001}, {'sunny': 0.031103999999999993, 'rainy': 0.0034560000000000003}]


In [13]:
result

(0.031103999999999993, ['sunny', 'sunny', 'sunny'])